# **05. Generalize Model**

**Topic**: Generalize your Model by adjusting the validation method, preprocessing, feature engineering, and so on.

In [1]:
import numpy as np

data = np.random.rand(5000)
x = np.random.rand(10)

### **Cut our Dataset into usages for training, testing and validation**

模型的架構(多少層,多深), 這些參數稱為 **Hyperparameter** (超參數)
那像是神經網路的權重參數, 這些參數稱為 **Weight parameter** (權重參數)

我們都知道 Weight parameter 會隨我們訓練來調整, 那 Hyperparameter 怎麼辦呢?
這時就是我們 validation data 的功用了, 我們可以看 validation 的結果來調整我們的模型架構
但這樣調整也怕過多, 造成模型去擬合我們的 validation data, 這種現象稱為 **information leak** (資訊洩漏)
當然我們也不想用 test data 來調整我們的 hyperparameter, 讓我們的 model 去學習 test data 就失去意義了


這邊提供幾種 validation data 驗證方法
* Simple hold-out validation
* K-fold validation
* Iterated K-fold validation with shuffling

#### **Simple hold-out validation**

這種方法適合在你有很多資料的時候來使用, 概念很簡單,

我們就乖乖的切 3 種 data, 分別是 train, test, validation, 

而 val data 通常是從 train 切出去的, 切記 val 不能太少, 不然不具有統計意義

In [2]:
num_val_data = 1000

np.random.shuffle(data)

val_data = data[:num_val_data]
train_data = data[num_val_data:]

經過 val data 調整完我們的參數後, 會再把 val 和 train 做合併再一起訓練

In [3]:
data = np.concatenate([train_data, val_data])

#### **K-fold validation**

<img src='https://scikit-learn.org/stable/_images/grid_search_cross_validation.png' alt='by scikitlearn'>

這方法適用於資料少的情況

將我們的 train data 切成 K 折, 並以 K-1 折作為我們的 train data, 並以其中 1 折來作為我們的 val data,

而這會重複 K 次來進行, 那每次選的 val data 會輪流來當, 最後我們把這些 K 次的 validation scores 來取平均作為我們的參考依據

In [4]:
k = 4
num_val_data = len(data)//k

np.random.shuffle(data)

val_scores = []
for fold in range(k):
    val_data = data[ num_val_data*fold : num_val_data*(fold+1) ]
    train_data = np.concatenate([data[ :num_val_data*fold ], data[ num_val_data*(fold+1): ] ])
    
    # 用 model evaluate 來算出 val scores 加到我們的 list
    score = 88
    val_scores.append(score)

#### **Iterated K-fold validation with shuffling**

此方法適用於資料相對較少, 且需要盡可能精確的評估模型的情況

這是目前在 Kaggle 常使用的方法, 簡單來講就是進行很多次的 K fold, 而每次資料都必須進行 shuffle, 

假設進行 P 次, 那麼就是說進行 P*K 回的訓練, 這方法相對來說運算成本相當的高

#### **Validation 注意事項**

* **資料代表性**
    * 假設我們要判斷 0～9 的資料, 我們不能把 label 0~7 為訓練, 8～9 為val, 這樣訓練就不具代表性, 因此我們需要 **randomly shuffle** 來處理資料
* **時間方向性**
    * 當處理時間序的資料, 我們界不能隨機取了, 因為這樣會造成 temporal leak (時間漏失) 發生時間錯位的狀況, 因此我們要確保 test data 都在 train data 之後
* **資料重複現象**
    * 有時候同樣的 label 及 feature 會出現在 train 及 val data 兩次, 這樣會造成表現不可信的狀況產生, 我們要避免

### **Preprocessing & Feature engineering**

#### **Data vectorization**

我們所用的 input 及 label 都必須將它轉換成 float 的張量來做運算, 這步驟就稱為 **Data vectorization**

#### **Normalization**

ex: 像是處理圖片的資料時, 我們必須將所有的值 / 255, 這是因為值得分佈範圍是 0～255, 我們透過 Normalization 的方式將範圍定在 0～1

這麼做是為了避免特徵彼此之間的影響會過大, 導致神經網路難以學習, 因此我們在做 Normalization 時, 記得每項特徵的數值範圍都應大致相同<br>
像是我們也常定義 Normalization 的分佈為:
* mean = 0
* std = 1

那我們可以進行以下操作

In [5]:
x -= x.mean(axis=0)
x /= x.std(axis=0)

#### **Missing Value**

資料中有 null出現的時候, 通常在 NN 中, 遇到缺失值我們會直接補 0, 前提是 0 比須不曾被定義於原本的資料中(不具參考意義）

這樣在訓練的時候 NN 就會去學習遇到 0 這個特徵時, 就知道他是缺失值,會自動去忽略它 

##### **Hint:**
如果說今天test data 裡有缺失值, 而 train data 沒有, 那我們就要為 train data 多加幾筆 missing value 出來<br>
不然你的 model 根本不知道遇到 missing value 該怎麼做<br>
簡單來做就是複製一些正常的樣本, 然後刪除幾的其中幾個特徵這樣

#### **Feature engineering**

特徵工程是透過我們自己的知識來為 input data 做 preprocessing, 這讓我們的 NN 能夠更有效率地來學習<br>
舉例來說,今天我們想解決的問題是: 用時鐘的照片來判斷幾點幾分<br>
如果我們直接將 img 丟進model來學習, 這會是相當沒效率又號效能的,<br>
但如果我們今天以時鐘中心為原點, 我們分別找出時針及分針的座標,接下來我們再來計算時針及分針所產生的夾角,<br>
我們把 img 轉成 夾角 這個動作就是所謂的特徵工程,<br>
這樣不僅節省我們的資料空間 (input是夾角的數值,不再是一個 image)<br>
也可以更有效率地去訓練我們的 model (資料少, 但訊息更明確)

### **Overfitting & Underfitting**

Machine Learning 基本上就是在 Optimization 及 Generalization 之間的拉鋸戰,<br>
Optimization 指在 train data 的表現 ; Generalization 指在未知data上的表現<br>
在訓練我們的 model 時, 一開始 train 和 validation data 間的表現會相似,<br>
因為 model 還沒有學習過多的 train data 特徵,<br>
而經過一段時間後, train data 會越來越好, 而 validation 的資料表現就漸漸趨於平緩,甚至反彈,<br>
這是因為我們的 model 學習過多的 train data 模式,就連一些微小的雜七雜八特徵都學進去了, <br>
這樣讓未知的資料集可能不適用這個 model,因為這 model 後來變成單純 for train data 的感覺,<br>
這過程稱之為 **Overfitting**<br>

--- 
想解決這個問題, 我們可以用兩種方式:
* Find More data ---> 避免以管窺天,當井底之蛙
* 調整 model 的複雜程度(more easy), 限制儲存資訊量  ---> 讓他不學那麼多特徵, 不學雜七雜八, 就更 generalize

像這一類避免 overfitting 的方式稱之為 **Regularization**

---
而 **Underfitting** 就是指 training 的 loss 無法降低，預測的準確率很低<br>
這通常發生在你的 NN 架構太簡單了, Model 無法去學習<br>
因此我們的資訊量找到一個平衡

**以下介紹幾種方法來改善 Overfitting**
* 縮減神經網路大小
* Weight Regularization
* Dropout

#### Example

##### 準備好我們的 data

In [6]:
from keras.datasets import imdb
import numpy as np

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

def vectorize_sequences(sequences, dimension=10000):
    
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

x_train = vectorize_sequences(train_data)

x_test = vectorize_sequences(test_data)

y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

Using TensorFlow backend.


#### **縮減神經網路大小**

##### Origin Model

In [7]:
from keras import models
from keras import layers

original_model = models.Sequential()
original_model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))  #原始的為 16 個單元
original_model.add(layers.Dense(16, activation='relu'))
original_model.add(layers.Dense(1, activation='sigmoid'))

original_model.compile(optimizer='rmsprop',
                       loss='binary_crossentropy',
                       metrics=['acc'])

##### Smaller Model

In [8]:
smaller_model = models.Sequential()
smaller_model.add(layers.Dense(4, activation='relu', input_shape=(10000,)))  #改成容量較低的 4 個單元
smaller_model.add(layers.Dense(4, activation='relu'))
smaller_model.add(layers.Dense(1, activation='sigmoid'))

smaller_model.compile(optimizer='rmsprop',
                      loss='binary_crossentropy',
                      metrics=['acc'])

##### Complete Model

In [9]:
bigger_model = models.Sequential()
bigger_model.add(layers.Dense(512, activation='relu', input_shape=(10000,)))  #改以更高容量的 512 個輸出單位
bigger_model.add(layers.Dense(512, activation='relu'))
bigger_model.add(layers.Dense(1, activation='sigmoid'))

bigger_model.compile(optimizer='rmsprop',
                     loss='binary_crossentropy',
                     metrics=['acc'])

##### Fit

In [10]:
original_hist = original_model.fit(x_train, y_train,
                                   epochs=20,
                                   batch_size=512,
                                   verbose=1,
                                   validation_data=(x_test, y_test))

smaller_model_hist = smaller_model.fit(x_train, y_train,
                                       epochs=20,
                                       batch_size=512,
                                       verbose=2,
                                       validation_data=(x_test, y_test))

bigger_model_hist = bigger_model.fit(x_train, y_train,
                                     epochs=20,
                                     batch_size=512,
                                     verbose=2,
                                     validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 8s 316us/step - loss: 0.4523 - acc: 0.8201 - val_loss: 0.3374 - val_acc: 0.8769
Epoch 2/20
25000/25000 [==============================] - 5s 184us/step - loss: 0.2530 - acc: 0.9102 - val_loss: 0.2877 - val_acc: 0.8864
Epoch 3/20
25000/25000 [==============================] - 5s 183us/step - loss: 0.1977 - acc: 0.9286 - val_loss: 0.2811 - val_acc: 0.8890
Epoch 4/20
25000/25000 [==============================] - 5s 183us/step - loss: 0.1657 - acc: 0.9416 - val_loss: 0.2961 - val_acc: 0.8826
Epoch 5/20
25000/25000 [==============================] - 5s 184us/step - loss: 0.1470 - acc: 0.9469 - val_loss: 0.3125 - val_acc: 0.8802
Epoch 6/20
25000/25000 [==============================] - 5s 186us/step - loss: 0.1275 - acc: 0.9558 - val_loss: 0.3383 - val_acc: 0.8749
Epoch 7/20
25000/25000 [==============================] - 5s 184us/step - loss: 0.1142 - acc: 0.9598 - val_loss: 0.3577 - 

##### Plot

In [11]:
epochs = range(1, 21)
original_val_loss = original_hist.history['val_loss']
smaller_model_val_loss = smaller_model_hist.history['val_loss']
bigger_model_val_loss = bigger_model_hist.history['val_loss']

In [12]:
import matplotlib.pyplot as plt

plt.plot(epochs, original_val_loss, 'b-', label='Original model')
plt.plot(epochs, bigger_model_val_loss, 'r-', label='Bigger model')
plt.plot(epochs, smaller_model_val_loss, 'g-', label='Smaller model')
plt.xlabel('Epochs')
plt.ylabel('Validation loss')
plt.legend()

plt.show()

<Figure size 640x480 with 1 Axes>

#### **Weight Regularization**

做法上是在原本的 loss function 中加上 Cost 項目, 讓較大的權重值在 Cost 上表現更明顯,<br>
這樣最後在調整參數時就優先處理他們, 抑制他們的表現,<br> 
這樣我們就是以較小的權重值來調整我們的 model,<br>
換句話說, 以較簡單的方式(less entropy)來解決我們的問題(調整我們的 model),這樣方式所適用的範圍會比複雜的方式更通用

----------------------

兩種常用方式來進行 Regularization:
* L1: 和 abs(weight parameters) 成正比
* L2: 和 pow(weight parameters, 2) 成正比

看以下紅框框就好, 前面 loss function 會變

<img src='https://miro.medium.com/max/2546/1*zMLv7EHYtjfr94JOBzjqTA.png' alt='l1_l2'>

##### **Example**

In [13]:
from keras import regularizers

# 這邊 l 是給予 lamda 的值
L1 = regularizers.l1(l=0.0001)         # l1
L2 = regularizers.l2(l=0.0001)         # l2
L1_2 = regularizers.l1_l2(l1=0.01,l2=0.0001)    # l1 + l2

In [14]:
# 使用

l2_model = models.Sequential()
l2_model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001),  #加入 L2 權重常規化並將學習率設為 0.001 
                          activation='relu', input_shape=(10000,)))
l2_model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001),
                          activation='relu'))
l2_model.add(layers.Dense(1, activation='sigmoid'))

#### **Dropout**

Dropout 是在 train 的時候, 隨機把某幾個數值的輸出歸 0, 而在 test 的時候就不會

這樣做就像是給 layer 加入一些**雜訊**, 有了這些雜訊就能避免神經網路去死背 data 的特徵

In [15]:
dpt_model = models.Sequential()
dpt_model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
dpt_model.add(layers.Dropout(0.5))
dpt_model.add(layers.Dense(16, activation='relu'))
dpt_model.add(layers.Dropout(0.5))
dpt_model.add(layers.Dense(1, activation='sigmoid'))

dpt_model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=['acc'])